# TCN Training Only (Clean)

This notebook is for **training only**.
It uses isolated `train_*` variables and a Sharpe-based checkpoint policy.

## 1) Connect to Colab VM and Sync Repo
Run this first.

In [1]:
# Fresh-start cleanup cell (run before importing project modules)
import gc
import shutil
import subprocess
import sys
from pathlib import Path

TRAIN_REPO_URL = "https://github.com/Dave-DKings/tape_tcn_project.git"
TRAIN_REPO_DIR = Path("/content/adaptive_portfolio_rl")

# 1) Sync repo to latest main
if not (TRAIN_REPO_DIR / ".git").exists():
    subprocess.run(["git", "clone", TRAIN_REPO_URL, str(TRAIN_REPO_DIR)], check=True)

subprocess.run(["git", "-C", str(TRAIN_REPO_DIR), "fetch", "origin"], check=True)
subprocess.run(["git", "-C", str(TRAIN_REPO_DIR), "reset", "--hard", "origin/main"], check=True)

# 2) Remove old experiment outputs/checkpoints/cached data
purge_paths = [
    TRAIN_REPO_DIR / "tcn_fusion_results",
    TRAIN_REPO_DIR / "tcn_results",
    TRAIN_REPO_DIR / "tcn_att_results",
    TRAIN_REPO_DIR / "output_logs",
    TRAIN_REPO_DIR / "data" / "phase1_preparation_artifacts",
    TRAIN_REPO_DIR / "data" / "master_features_NORMALIZED.csv",
    TRAIN_REPO_DIR / "data" / "daily_ohlcv_assets.csv",              # forces fresh OHLCV download
    TRAIN_REPO_DIR / "data" / "processed_daily_macro_features.csv",   # forces fresh macro cache build
]

deleted = []
for p in purge_paths:
    if p.is_dir():
        shutil.rmtree(p, ignore_errors=True)
        deleted.append(str(p))
    elif p.is_file():
        p.unlink(missing_ok=True)
        deleted.append(str(p))

# 3) Remove Python/Jupyter cache folders
for cache_dir in TRAIN_REPO_DIR.rglob("__pycache__"):
    shutil.rmtree(cache_dir, ignore_errors=True)
for ckpt_dir in TRAIN_REPO_DIR.rglob(".ipynb_checkpoints"):
    shutil.rmtree(ckpt_dir, ignore_errors=True)

# 4) Clear loaded project modules from kernel memory
for mod in list(sys.modules.keys()):
    if mod.startswith("src.") or mod.startswith("src_"):
        del sys.modules[mod]
gc.collect()

print("✅ Fresh start complete")
print(f"Repo: {TRAIN_REPO_DIR}")
print(f"Deleted paths: {len(deleted)}")
for d in deleted:
    print(" -", d)

✅ Fresh start complete
Repo: /content/adaptive_portfolio_rl
Deleted paths: 3
 - /content/adaptive_portfolio_rl/tcn_fusion_results
 - /content/adaptive_portfolio_rl/data/master_features_NORMALIZED.csv
 - /content/adaptive_portfolio_rl/data/daily_ohlcv_assets.csv


In [2]:
#from pathlib import Path
import os

root = Path("/content/adaptive_portfolio_rl")
print("Exists:", root.exists())
print("CWD:", os.getcwd())

print("\nTop-level:")
for p in sorted(root.iterdir()):
    kind = "DIR " if p.is_dir() else "FILE"
    print(f" - [{kind}] {p.name}")

# Quick check for outputs/caches you expected to be deleted
targets = [
    "tcn_fusion_results",
    "tcn_results",
    "tcn_att_results",
    "output_logs",
    "data/phase1_preparation_artifacts",
    "data/master_features_NORMALIZED.csv",
    "data/daily_ohlcv_assets.csv",
    "data/processed_daily_macro_features.csv",
]
print("\nTarget paths:")
for t in targets:
    p = root / t
    print(f" - {t}: {'EXISTS' if p.exists() else 'MISSING'}")


Exists: True
CWD: /content

Top-level:
 - [DIR ] .git
 - [FILE] .gitignore
 - [FILE] RL Portfolio Optimization Feature Engineering.md
 - [FILE] RL_Portfolio_Optimization_Feature_Engineering.ipynb
 - [FILE] USAGE_GUIDE_ACTUARIAL.py
 - [FILE] __init__.py
 - [FILE] convert_md_to_ipynb.py
 - [DIR ] data
 - [FILE] data1.zip
 - [DIR ] data_exports
 - [FILE] debug_attention_weights.py
 - [DIR ] eval
 - [DIR ] paper
 - [FILE] requirements.txt
 - [FILE] run_tcn_eval.py
 - [DIR ] src
 - [FILE] tcn_architecture_analysis.ipynb
 - [DIR ] tcn_documentation
 - [FILE] tcn_evaluation_only.ipynb
 - [FILE] technical_deep_dive_presentation.ipynb
 - [DIR ] tests
 - [FILE] traditional_portfolio_benchmarks.ipynb
 - [DIR ] training_scripts

Target paths:
 - tcn_fusion_results: MISSING
 - tcn_results: MISSING
 - tcn_att_results: MISSING
 - output_logs: MISSING
 - data/phase1_preparation_artifacts: MISSING
 - data/master_features_NORMALIZED.csv: MISSING
 - data/daily_ohlcv_assets.csv: MISSING
 - data/processed_

In [ ]:
#!find /content/adaptive_portfolio_rl -maxdepth 3 | head -n 300

In [ ]:
# Install project requirements in Colab VM
#import subprocess, sys
#from pathlib import Path

REPO_DIR = Path("/content/adaptive_portfolio_rl")
REQ_FILE = REPO_DIR / "requirements.txt"

if not REQ_FILE.exists():
    raise FileNotFoundError(f"Missing requirements file: {REQ_FILE}")

print("Using python:", sys.executable)
subprocess.run([sys.executable, "-m", "pip", "install", "--upgrade", "pip", "setuptools", "wheel"], check=True)
subprocess.run([sys.executable, "-m", "pip", "install", "-r", str(REQ_FILE)], check=True)

print("✅ Requirements installed")


In [3]:
import numpy, pandas, tensorflow
print("numpy", numpy.__version__)
print("pandas", pandas.__version__)
print("tensorflow", tensorflow.__version__)

numpy 2.0.2
pandas 2.2.2
tensorflow 2.19.0


## 2) Imports

In [4]:
import os, sys
from pathlib import Path

REPO_DIR = Path("/content/adaptive_portfolio_rl")

if not REPO_DIR.exists():
    raise FileNotFoundError(f"Repo not found: {REPO_DIR}")

# Set working directory
os.chdir(REPO_DIR)

# Add repo root to Python path
if str(REPO_DIR) not in sys.path:
    sys.path.insert(0, str(REPO_DIR))

print("cwd:", os.getcwd())
print("sys.path[0]:", sys.path[0])

cwd: /content/adaptive_portfolio_rl
sys.path[0]: /content/adaptive_portfolio_rl


In [5]:
from copy import deepcopy
from pathlib import Path

import pandas as pd

from src.config import get_active_config
from src.csv_logger import CSVLogger
from src.notebook_helpers.tcn_phase1 import prepare_phase1_dataset, run_experiment6_tape

## 3) Base Config and Dataset Prep

In [6]:
# ------------------------------------------------------------------
# Feature lock from CORE project pipeline (no metadata/manifest)
# ------------------------------------------------------------------
from src.data_utils import DataProcessor


def build_core_active_feature_columns(cfg):
    # Active list as defined by core config + current selection rules.
    probe = DataProcessor(cfg)
    return list(dict.fromkeys(probe.get_feature_columns("phase1")))


def apply_core_feature_lock(cfg, active_feature_columns):
    # Compute full candidate pool with selection filter temporarily disabled,
    # then enforce active-only by setting disabled_features = full - active.
    probe_cfg = deepcopy(cfg)
    probe_fp = probe_cfg.setdefault("feature_params", {})
    probe_fs = probe_fp.setdefault("feature_selection", {})
    probe_fs["disable_features"] = False
    probe_fs["disabled_features"] = []

    probe = DataProcessor(probe_cfg)
    core_all_cols = list(dict.fromkeys(probe.get_feature_columns("phase1")))

    active_set = set(active_feature_columns)
    disabled = sorted([c for c in core_all_cols if c not in active_set])

    fp = cfg.setdefault("feature_params", {})
    fs = fp.setdefault("feature_selection", {})
    fs["disable_features"] = True
    fs["disabled_features"] = disabled

    return core_all_cols, disabled


In [7]:
TRAIN_RANDOM_SEED = 42

train_config = deepcopy(get_active_config("phase1"))

# Optional: override analysis horizon
# train_config["ANALYSIS_END_DATE"] = "2025-09-01"

# Build active features from core project files/pipeline.
train_active_feature_columns = build_core_active_feature_columns(train_config)
_, train_disabled_features = apply_core_feature_lock(train_config, train_active_feature_columns)

print("✅ Core feature lock applied")
print("   active_feature_columns:", len(train_active_feature_columns))
print("   disabled_features:", len(train_disabled_features))

# Force fresh dataset build and market data re-download
if "train_phase1_data" in globals():
    del train_phase1_data

train_phase1_data = prepare_phase1_dataset(
    train_config,
    force_download=True,
    preparation_artifacts_dir="/content/adaptive_portfolio_rl/data_exports",
)


✅ Core feature lock applied
   active_feature_columns: 50
   disabled_features: 25
📊 Loading raw market data...
   ✅ Raw data shape: (55107, 7)
   ✅ Date range: 2003-09-02 00:00:00 → 2025-08-29 00:00:00

🔧 Computing multi-horizon log returns: [1, 5, 10, 21]
   ✅ Shape after returns: (54897, 11)

📈 Calculating 21-day rolling statistics

🧮 Computing technical indicators

📊 Computing dynamic covariance features

🎯 Adding regime awareness features
   ✅ Master DF shape: (54897, 47)
   ✅ Total features: 47

📊 Integrating fundamental features (if enabled)...
   ✅ Fundamental columns in dataset: 6 (enabled=True)
   🧾 Sample fundamental cols: ['Fundamental_FCFE_Delta', 'Fundamental_Revenue_Delta', 'Fundamental_NCFO_Delta', 'Fundamental_FCFE_Sign', 'Fundamental_Staleness_Days', 'Fundamental_Staleness_Quarters']

📊 Integrating macroeconomic features (if enabled)...
   ✅ Macro features added - 43 columns: ['EFFR_diff', 'EFFR_zscore', 'SOFR_level', 'SOFR_diff', 'FEDFUNDS_diff', 'FEDFUNDS_zscore', '


💾 Saving NORMALISED master dataframe to '/content/adaptive_portfolio_rl/data/master_features_NORMALIZED.csv'

💾 Saved preparation artifacts:
   raw OHLCV: /content/adaptive_portfolio_rl/data_exports/phase1_prep_20260221_074148_raw_ohlcv.csv
   full engineered: /content/adaptive_portfolio_rl/data_exports/phase1_prep_20260221_074148_feature_engineered_full.csv
   analysis-window engineered: /content/adaptive_portfolio_rl/data_exports/phase1_prep_20260221_074148_feature_engineered_analysis_window.csv
   normalized master: /content/adaptive_portfolio_rl/data_exports/phase1_prep_20260221_074148_feature_engineered_normalized.csv
   train normalized: /content/adaptive_portfolio_rl/data_exports/phase1_prep_20260221_074148_train_normalized.csv
   test normalized: /content/adaptive_portfolio_rl/data_exports/phase1_prep_20260221_074148_test_normalized.csv
   scalers: /content/adaptive_portfolio_rl/data_exports/phase1_prep_20260221_074148_scalers.joblib
   audit report: /content/adaptive_portfoli

In [8]:
print("Train shape:", train_phase1_data.train_df.shape)
print("Test shape:", train_phase1_data.test_df.shape)

cols = train_phase1_data.train_df.columns
print("Total columns:", len(cols))

# quick sanity for common redundant groups
dup_like = [c for c in cols if c.endswith("_raw") or c.endswith("_unscaled")]
print("Potential redundant raw/unscaled cols:", len(dup_like))
print(dup_like[:20])

Train shape: (43867, 105)
Test shape: (11030, 105)
Total columns: 105
Potential redundant raw/unscaled cols: 0
[]


In [9]:
cols

Index(['Date', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Volume',
       'LogReturn_1d', 'LogReturn_5d', 'LogReturn_10d',
       ...
       'VIX_zscore', 'CrossSectional_ZScore_LogReturn_1d',
       'Residual_Momentum_21', 'Volume_Percentile_63', 'YieldCurve_Spread',
       'YieldCurve_Inverted_Flag', 'ShortTerm_Reversal_5', 'VolOfVol_63',
       'Beta_to_Market', 'OBV_Delta_Norm_21'],
      dtype='object', length=105)

In [10]:
used = set(train_phase1_data.data_processor.get_feature_columns("phase1"))
disabled = set(train_config["feature_params"]["feature_selection"]["disabled_features"])

print("Used feature count:", len(used))
print("Disabled that still in used:", sorted(disabled & used))  # should be []
print("VIX_zscore used?", "VIX_zscore" in used)

Used feature count: 73
Disabled that still in used: []
VIX_zscore used? False


In [11]:
base_cols = ["Date", "Ticker", "Open", "High", "Low", "Close", "Volume"]
keep = [c for c in base_cols + list(used) if c in train_phase1_data.master_df.columns]

train_phase1_data.master_df = train_phase1_data.master_df[keep].copy()
train_phase1_data.train_df = train_phase1_data.train_df[keep].copy()
train_phase1_data.test_df  = train_phase1_data.test_df[keep].copy()

## 4) Training Overrides (Sharpe-Only Checkpoint Policy)

This policy keeps only Sharpe-threshold high-watermark checkpointing (`>= 0.5`) and disables rare/step/periodic/TAPE checkpoint routes.

In [12]:
from copy import deepcopy

train_config = deepcopy(train_config)  # or deepcopy(config) if that is your active object

tp = train_config["training_params"]
ap = train_config["agent_params"]
ppo = ap["ppo_params"]
env = train_config["environment_params"]

# Core setup
tp["max_total_timesteps"] = 100_000
tp["timesteps_per_ppo_update"] = 384  # fallback
tp["timesteps_per_ppo_update_schedule"] = [
    {"threshold": 0, "timesteps_per_update": 384},
    {"threshold": 30_000, "timesteps_per_update": 512},
]
tp["batch_size_ppo_schedule"] = [
    {"threshold": 0, "batch_size": 128},
    {"threshold": 30_000, "batch_size": 128},
]

# Episode-length curriculum (ENABLE)
tp["use_episode_length_curriculum"] = True
tp["episode_length_curriculum_schedule"] = [
    {"threshold": 0, "limit": 252},
    {"threshold": 10_000, "limit": 504},
    {"threshold": 25_000, "limit": 756},
    {"threshold": 40_000, "limit": None},
]

# PPO KL management
ppo["target_kl"] = 0.020
ppo["kl_stop_multiplier"] = 1.25
ppo["minibatches_before_kl_stop"] = 2
ppo["policy_clip"] = 0.08
ppo["num_ppo_epochs"] = 1
ppo["max_grad_norm"] = 0.30

# LR + entropy
ppo["actor_lr"] = 8e-6
ppo["critic_lr"] = 1.2e-4
ppo["entropy_coef"] = 0.0015

# IMPORTANT: override training-level actor LR schedule (otherwise defaults win)
tp["actor_lr_schedule"] = [
    {"threshold": 0, "lr": 8e-6},
    {"threshold": 40_000, "lr": 7e-6},
    {"threshold": 70_000, "lr": 6e-6},
]

# Dirichlet controls
ap["dirichlet_alpha_activation"] = "elu"
ap["dirichlet_logit_temperature"] = 1.20
ap["dirichlet_alpha_cap"] = 40.0
ap["dirichlet_epsilon"] = {"max": 0.9, "min": 0.3}

# Keep execution/turnover controls
tp["action_execution_beta_curriculum"] = {
    0: 0.20,
    30_000: 0.35,
}
tp["turnover_penalty_curriculum"] = {
    0: 1.50,
    10_000: 2.00,
    25_000: 2.50,
    40_000: 3.00,
}
env["target_turnover"] = 0.35
env["turnover_penalty_scalar"] = 1.50
env["transaction_cost_pct"] = 0.001

# Keep terminal reward scale aligned to your intended setup
env["tape_terminal_scalar"] = 6.0
env["tape_terminal_clip"] = 6.0

# Sharpe-only checkpoint policy
tp["high_watermark_checkpoint_enabled"] = True
tp["high_watermark_sharpe_threshold"] = 0.5
tp["step_sharpe_checkpoint_enabled"] = False
tp["periodic_checkpoint_every_steps"] = 0
tp["tape_checkpoint_threshold"] = 999.0
tp["rare_checkpoint_params"] = {"enable": False}

print("✅ KL tuning + Sharpe-only checkpoint policy applied")
print("target_kl:", ppo["target_kl"])
print("rollout schedule:", tp["timesteps_per_ppo_update_schedule"])
print("batch schedule:", tp["batch_size_ppo_schedule"])
print("episode_length_curriculum:", tp["use_episode_length_curriculum"], tp["episode_length_curriculum_schedule"])
print("actor_lr_schedule:", tp["actor_lr_schedule"])
print("tape_terminal:", env["tape_terminal_scalar"], env["tape_terminal_clip"])
print("dirichlet:", ap["dirichlet_alpha_activation"], ap["dirichlet_logit_temperature"], ap["dirichlet_alpha_cap"], ap["dirichlet_epsilon"])
print("high_watermark_sharpe_threshold:", tp["high_watermark_sharpe_threshold"])


✅ KL tuning + Sharpe-only checkpoint policy applied
target_kl: 0.02
rollout schedule: [{'threshold': 0, 'timesteps_per_update': 384}, {'threshold': 30000, 'timesteps_per_update': 512}]
batch schedule: [{'threshold': 0, 'batch_size': 128}, {'threshold': 30000, 'batch_size': 128}]
episode_length_curriculum: True [{'threshold': 0, 'limit': 252}, {'threshold': 10000, 'limit': 504}, {'threshold': 25000, 'limit': 756}, {'threshold': 40000, 'limit': None}]
actor_lr_schedule: [{'threshold': 0, 'lr': 8e-06}, {'threshold': 40000, 'lr': 7e-06}, {'threshold': 70000, 'lr': 6e-06}]
tape_terminal: 6.0 6.0
dirichlet: elu 1.2 40.0 {'max': 0.9, 'min': 0.3}
high_watermark_sharpe_threshold: 0.5


In [13]:
#tp = train_config["training_params"]  # or config["training_params"]

# Practical logging cadence
tp["update_log_interval"] = 5
tp["alpha_diversity_log_interval"] = 5

# Alpha-stuck warning controls
tp["alpha_diversity_warning_after_updates"] = 300
tp["alpha_diversity_warning_std_threshold"] = 0.25

tp["log_step_diagnostics"] = True  

## 5) Run Training

In [14]:
RUN_TRAINING = True

if RUN_TRAINING:
    tp = train_config["training_params"]
    print("🚀 Starting training")
    print("Architecture:", train_config["agent_params"].get("actor_critic_type"))
    print("max_total_timesteps:", tp["max_total_timesteps"])

    train_experiment6 = run_experiment6_tape(
        phase1_data=train_phase1_data,
        config=train_config,
        random_seed=TRAIN_RANDOM_SEED,
        csv_logger_cls=CSVLogger,
        use_covariance=True,
        architecture=train_config["agent_params"].get("actor_critic_type"),
        timesteps_per_update=tp.get("timesteps_per_ppo_update", 384),
        max_total_timesteps=tp["max_total_timesteps"],
    )

    print("✅ Training complete")
    print("checkpoint_prefix:", train_experiment6.checkpoint_path)
else:
    print("ℹ️ RUN_TRAINING=False")

🚀 Starting training
Architecture: TCN_FUSION
max_total_timesteps: 100000

EXPERIMENT 6: TCN_FUSION Enhanced + TAPE Three-Component
Architecture: TCN + Fusion
Results root: /content/adaptive_portfolio_rl/tcn_fusion_results
Working dir: /content/adaptive_portfolio_rl
Covariance Features: Yes
🎯 REWARD SYSTEM: TAPE (Three-Component v3)
   Profile: BalancedGrowth
   Daily: Base + DSR/PBRS + Turnover_Proximity
   Terminal: mode=signed | baseline=0.20 | scalar=6.0 (clipped ±6.0)
   Gate A: enabled (Sharpe ≤ 0.00 or MDD ≥ 25.0% -> force non-positive terminal bonus)
   Neutral Band: enabled (±0.020 around baseline)
   🔄 Profile Manager: disabled (static profile only)
🎲 Experiment Seed: 6042 (Base: 42, Offset: 6000)
✅ Features: Enhanced (includes 3 covariance eigenvalues)
   Eigenvalues: ['Covariance_Eigenvalue_0', 'Covariance_Eigenvalue_1', 'Covariance_Eigenvalue_2']
   Train shape: (43867, 80)
   Test shape: (11030, 80)

🏗️ Creating THREE-COMPONENT TAPE v3 environments (with curriculum)...
   

   🎯 Episode 2: TAPE Score = 0.5674 (bonus: +2.76 → +2.76)
      💾 Sharpe-threshold checkpoint saved: /content/adaptive_portfolio_rl/tcn_fusion_results/high_watermark_checkpoints/exp6_tape_hw_ep00002_shp1p209_actor.weights.h5 (Sharpe=1.209)
   🎯 Episode 3: TAPE Score = 0.6686 (bonus: +3.51 → +3.51)
      💾 Sharpe-threshold checkpoint saved: /content/adaptive_portfolio_rl/tcn_fusion_results/high_watermark_checkpoints/exp6_tape_hw_ep00003_shp1p822_actor.weights.h5 (Sharpe=1.822)
   🎯 Episode 4: TAPE Score = 0.2610 (bonus: -0.46 → -0.46)
      🚦 Gate A applied: Sharpe=0.517, MDD=30.36%
      💾 Sharpe-threshold checkpoint saved: /content/adaptive_portfolio_rl/tcn_fusion_results/high_watermark_checkpoints/exp6_tape_hw_ep00004_shp0p517_actor.weights.h5 (Sharpe=0.517)


   🎯 Episode 5: TAPE Score = 0.5996 (bonus: +3.00 → +3.00)
      💾 Sharpe-threshold checkpoint saved: /content/adaptive_portfolio_rl/tcn_fusion_results/high_watermark_checkpoints/exp6_tape_hw_ep00005_shp1p406_actor.weights.h5 (Sharpe=1.406)
   🎯 Episode 6: TAPE Score = 0.7532 (bonus: +4.15 → +4.15)
      💾 Sharpe-threshold checkpoint saved: /content/adaptive_portfolio_rl/tcn_fusion_results/high_watermark_checkpoints/exp6_tape_hw_ep00006_shp3p251_actor.weights.h5 (Sharpe=3.251)
   🎯 Episode 7: TAPE Score = 0.4563 (bonus: +1.92 → +1.92)
      💾 Sharpe-threshold checkpoint saved: /content/adaptive_portfolio_rl/tcn_fusion_results/high_watermark_checkpoints/exp6_tape_hw_ep00007_shp0p778_actor.weights.h5 (Sharpe=0.778)
🔄 Update 5/216 | Step 1,920/100,000 | Episode 7 | Time: 283.5s
   📊 Metrics: Return=+10.59% | Sharpe=0.778 | DD=10.59% | Turnover=10.55%
   🎚️ Intra-Step TAPE: potential=0.2497 | delta_reward=+0.0000
   🎯 Profile: BalancedGrowth
   🧠 Training: actor_loss=0.0206 | critic_loss=0

   🎯 Episode 10: TAPE Score = 0.3074 (bonus: +0.81 → +0.81)
   🎯 Episode 11: TAPE Score = 0.5608 (bonus: +2.71 → +2.71)
      💾 Sharpe-threshold checkpoint saved: /content/adaptive_portfolio_rl/tcn_fusion_results/high_watermark_checkpoints/exp6_tape_hw_ep00011_shp1p233_actor.weights.h5 (Sharpe=1.233)
   🎯 Episode 12: TAPE Score = 0.2435 (bonus: -0.33 → -0.33)
      🚦 Gate A applied: Sharpe=-0.176, MDD=17.92%


   🎯 Episode 13: TAPE Score = 0.5226 (bonus: +2.42 → +2.42)
      💾 Sharpe-threshold checkpoint saved: /content/adaptive_portfolio_rl/tcn_fusion_results/high_watermark_checkpoints/exp6_tape_hw_ep00013_shp1p106_actor.weights.h5 (Sharpe=1.106)
   🎯 Episode 14: TAPE Score = 0.2574 (bonus: +0.43 → +0.43)
   🎯 Episode 15: TAPE Score = 0.6667 (bonus: +3.50 → +3.50)
      💾 Sharpe-threshold checkpoint saved: /content/adaptive_portfolio_rl/tcn_fusion_results/high_watermark_checkpoints/exp6_tape_hw_ep00015_shp1p810_actor.weights.h5 (Sharpe=1.810)


...

🔄 Update 210/216 | Step 97,408/100,000 | Episode 113 | Time: 13701.3s
   📊 Metrics: Return=+93.12% | Sharpe=0.304 | DD=39.37% | Turnover=21.38%
   🎚️ Intra-Step TAPE: potential=0.2477 | delta_reward=+0.0000
   🎯 Profile: BalancedGrowth
   🧠 Training: actor_loss=0.0234 | critic_loss=0.2130 | mean_adv=0.0000
   🧮 Loss Detail: critic_scaled=0.1065 | risk_aux_total=0.0000 | sharpe_proxy=0.0000 | sharpe_loss=0.0000 | mvo_loss=0.0000
   ⚙️ Optimizer: actor_lr=0.000006 | critic_lr=0.000120 | target_kl=0.0200 | rollout=512 | batch_size=128
   🔬 Alpha Diversity: mean=1.43 | std=0.30 | range=[0.81, 1.96]
   🎯 Episode 114: TAPE Score = 0.2449 (bonus: -0.34 → -0.34)
      🚦 Gate A applied: Sharpe=0.396, MDD=39.37%

📚 EPISODE HORIZON UPDATE at 97,920 steps:
   Episode horizon set to full dataset



📚 EPISODE HORIZON UPDATE at 98,432 steps:
   Episode horizon set to full dataset
   🎯 Episode 115: TAPE Score = 0.3140 (bonus: -0.86 → -0.86)
      🚦 Gate A applied: Sharpe=0.702, MDD=30.05%
      💾 Sharpe-threshold checkpoint saved: /content/adaptive_portfolio_rl/tcn_fusion_results/high_watermark_checkpoints/exp6_tape_hw_ep00115_shp0p702_actor.weights.h5 (Sharpe=0.702)

📚 EPISODE HORIZON UPDATE at 98,944 steps:
   Episode horizon set to full dataset



📚 EPISODE HORIZON UPDATE at 99,456 steps:
   Episode horizon set to full dataset



📚 EPISODE HORIZON UPDATE at 99,968 steps:
   Episode horizon set to full dataset


🔄 Update 215/216 | Step 99,968/100,000 | Episode 115 | Time: 14058.2s
   📊 Metrics: Return=+68.15% | Sharpe=0.702 | DD=30.05% | Turnover=21.01%
   🎚️ Intra-Step TAPE: potential=0.7037 | delta_reward=+0.0002
   🎯 Profile: BalancedGrowth
   🧠 Training: actor_loss=0.0625 | critic_loss=0.2703 | mean_adv=0.0000
   🧮 Loss Detail: critic_scaled=0.1351 | risk_aux_total=0.0000 | sharpe_proxy=0.0000 | sharpe_loss=0.0000 | mvo_loss=0.0000
   ⚙️ Optimizer: actor_lr=0.000006 | critic_lr=0.000120 | target_kl=0.0200 | rollout=512 | batch_size=128
   🔬 Alpha Diversity: mean=1.47 | std=0.37 | range=[0.42, 2.48]

📚 EPISODE HORIZON UPDATE at 100,000 steps:
   Episode horizon set to full dataset
🔄 Update 216/216 | Step 100,000/100,000 | Episode 115 | Time: 14066.5s
   📊 Metrics: Return=+78.52% | Sharpe=0.650 | DD=30.25% | Turnover=21.05%
   🎚️ Intra-Step TAPE: potential=0.7004 | delta_reward=+0.0002
   🎯 Profile: BalancedGrowth
   🧠 Training: actor_loss=0.0201 | critic_loss=0.3658 | mean_adv=-0.0000
   🧮 

## 6) Inspect Latest Training Logs

In [15]:
TRAIN_RESULTS_ROOT = Path("/content/adaptive_portfolio_rl/tcn_fusion_results")
TRAIN_LOGS_DIR = TRAIN_RESULTS_ROOT / "logs"

episodes_files = sorted(TRAIN_LOGS_DIR.glob("*episodes*.csv"), key=lambda p: p.stat().st_mtime, reverse=True)
if not episodes_files:
    print(f"No episodes CSV found in {TRAIN_LOGS_DIR}")
else:
    train_episodes_path = episodes_files[0]
    train_episodes_df = pd.read_csv(train_episodes_path)
    print("Episodes file:", train_episodes_path)
    print("Rows:", len(train_episodes_df))
    display(train_episodes_df.tail(20))

Episodes file: /content/adaptive_portfolio_rl/tcn_fusion_results/logs/Exp6_TCN_FUSION_Enhanced_TAPE_training_20260221_074438_episodes.csv
Rows: 44


,update,timestep,episode,elapsed_time,episode_return_pct,episode_sharpe,episode_sortino,episode_max_dd,episode_volatility,episode_win_rate,...,actor_grad_norm,critic_grad_norm,alpha_min,alpha_max,alpha_mean,ratio_mean,ratio_std,drawdown_lambda_peak,episode_length,termination_reason
24,125,53888,95,7588.848321,147.280347,0.368493,0.471990,45.234094,0.167654,53.197158,...,0.991537,0.276854,0.917434,2.327151,1.557742,1.022158,0.193112,5.000000,3904.0,data_exhausted
25,130,56448,96,7945.142123,238.893770,0.422210,0.528641,45.234094,0.178832,53.352243,...,1.336362,1.444798,0.940221,2.157194,1.522272,0.976395,0.183761,3.610326,3879.0,data_exhausted
26,135,59008,98,8301.014248,71.540151,0.716565,0.834350,31.673584,0.192868,55.942623,...,1.121846,0.524080,0.822288,2.389275,1.506629,1.001624,0.238133,0.118620,977.0,data_exhausted
27,140,61568,98,8660.969729,108.180734,0.318511,0.406986,46.303088,0.169908,52.979066,...,1.659853,0.940132,0.901950,2.202467,1.484410,0.998578,0.192684,0.118620,977.0,data_exhausted
28,145,64128,99,9024.957481,208.949415,0.407927,0.510961,46.303088,0.180258,53.620830,...,1.490246,2.463688,0.806937,2.074920,1.460417,1.012295,0.188959,5.000000,3688.0,data_exhausted
29,150,66688,100,9386.984236,181.483281,0.653732,0.821953,31.050019,0.154981,53.283145,...,1.200478,0.395522,0.757536,1.975055,1.417079,0.996568,0.202963,0.105585,2392.0,data_exhausted
30,155,69248,101,9752.312308,250.796114,0.406314,0.510433,48.938057,0.173097,53.635932,...,1.069966,0.180694,0.765731,2.005818,1.420008,1.005967,0.193391,5.000000,4209.0,data_exhausted
31,160,71808,102,10115.577685,218.562839,0.632750,0.810252,29.268378,0.156272,53.117025,...,0.910754,0.616523,0.425653,2.499219,1.458250,0.992416,0.238600,0.075213,2744.0,data_exhausted
32,165,74368,103,10475.935310,83.537822,0.591312,0.703680,32.117387,0.172298,54.012561,...,1.559918,1.165193,0.804836,2.058987,1.434285,0.992292,0.228332,0.145043,1434.0,data_exhausted
33,170,76928,104,10836.039549,104.755663,0.556044,0.671867,30.725870,0.160362,53.351064,...,1.423404,1.273925,0.416683,2.497421,1.463326,0.973909,0.262075,0.109403,1881.0,data_exhausted


In [16]:
train_episodes_df.columns

Index(['update', 'timestep', 'episode', 'elapsed_time', 'episode_return_pct',
       'episode_sharpe', 'episode_sortino', 'episode_max_dd',
       'episode_volatility', 'episode_win_rate', 'episode_turnover',
       'episode_turnover_pct', 'episode_return_skew', 'episode_calmar_ratio',
       'episode_omega_ratio', 'episode_ulcer_index', 'episode_cvar_5pct',
       'profile_name', 'turnover_scalar', 'terminal_drawdown_lambda',
       'terminal_drawdown_lambda_peak', 'terminal_drawdown_avg_excess',
       'terminal_drawdown_penalty_sum', 'snapshot_drawdown_lambda',
       'snapshot_drawdown_lambda_peak', 'snapshot_drawdown_current',
       'snapshot_drawdown_avg_excess', 'snapshot_drawdown_penalty_sum',
       'snapshot_drawdown_triggered', 'snapshot_drawdown_trigger_boundary',
       'snapshot_drawdown_target', 'snapshot_drawdown_tolerance',
       'snapshot_intra_step_tape_potential',
       'snapshot_intra_step_tape_delta_reward', 'drawdown_lambda',
       'tape_score', 'tape_bonus',

## 7) Export Results Folder (Optional)
Creates a zip for download from Colab VM.

In [17]:
from pathlib import Path
import subprocess

EXPORT_RESULTS_ZIP = True
EXPORT_PATH = Path("/content/tcn_fusion_results_export.zip")
ROOT = Path("/content/adaptive_portfolio_rl")

if EXPORT_RESULTS_ZIP:
    include_paths = [
        ROOT / "tcn_fusion_results",
        ROOT / "data" / "phase1_preparation_artifacts",
        ROOT / "data" / "master_features_NORMALIZED.csv",
    ]

    existing = [p for p in include_paths if p.exists()]
    if not existing:
        print("⚠️ Nothing to export.")
    else:
        if EXPORT_PATH.exists():
            EXPORT_PATH.unlink()

        rel_items = [str(p.relative_to(ROOT)) for p in existing]
        cmd = f"cd {ROOT} && zip -qr {EXPORT_PATH} " + " ".join(rel_items)
        subprocess.run(cmd, shell=True, check=True)

        print(f"✅ Created: {EXPORT_PATH}")
        print("Included:")
        for p in rel_items:
            print(" -", p)
else:
    print("ℹ️ EXPORT_RESULTS_ZIP=False")


✅ Created: /content/tcn_fusion_results_export.zip
Included:
 - tcn_fusion_results
 - data/master_features_NORMALIZED.csv


In [18]:
from google.colab import drive
drive.mount('/content/drive')

!cp /content/tcn_fusion_results_export.zip /content/drive/MyDrive/
print("✅ Copied to Drive: /content/drive/MyDrive/tcn_fusion_results_export.zip")

Mounted at /content/drive
✅ Copied to Drive: /content/drive/MyDrive/tcn_fusion_results_export.zip
